In [ ]:
import sys
sys.path.append('..')
sys.path.append('../..')

import pandas as pd

from src.tools import *
from src.database import Database
from src.values import Values
from src.prices.prices import Price

p = Price(delay=20)
price = p.price
db1 = Database('db1.db')

txs = db1.read('txs')

I valori di "in" e "out" di ogni transazione, calcolati in euro, potrebbero essere diversi tra loro perchè i tassi di cambio sono riportati come media giornaliera. 
Scegliamo di usare un valore solo (quello dell'asset più liquido), affinchè il conto globale delle plusvalenze risulti corretto.

Definiamo come "Value" di una transazione il valore IN o OUT dell'asset più liquido

In [ ]:
# totalmente discrezionale, inserire in lista gli asset reputati più liquidi, in ordine.
# Almeno IN o OUT di ogni transazione devono essere compresi in questa lista

isos = ['EUR','USD','USDT','BUSD', 'BTC', 'ETH', 'BNB', 'ALGO', 'DASH']
isos.reverse()

In [ ]:
pr = {}
for i, x in enumerate(isos):
    pr[x]=i
def prior_iso(r):  
    inn = r['in_iso']
    out = r['out_iso']
    if inn in pr and out in pr:
        return 'in' if pr[inn]>pr[out] else 'out'
    elif inn in pr and out not in pr:
        return 'in'
    elif inn not in pr and out in pr:
        return 'out'
    else:
        raise Exception('what?', r)
    
def istrade(r):
    inn = pd.notna(r['in']) & (r['in'] != 0)
    out = pd.notna(r['out']) & (r['out'] != 0)
    return True if inn & out else False

def noninisos(r):
    test1 =( r['in_iso'] in isos) 
    test2 = (r['out_iso'] in isos) 
    return ( not test1 ) & (not test2)

In [ ]:
trademask = txs.apply(istrade, axis=1)
noninisosmask = txs.apply(noninisos, axis=1)

verifichiamo che non esistano trades che non coinvolgano le coin "notevoli" di isos

In [ ]:

txs[trademask & noninisosmask] 

In [ ]:
txs[~trademask & (txs['type']=='trade')]

con trademask verifico che tutti i trades abbiano in & out 

In [ ]:
newcol = ['value', 'in_price', 'out_price', 'fee_price']
txs[newcol]=[None, None, None, None]

In [ ]:
newcol = txs.columns

In [ ]:
def valorize(r):
    #print(r)
    t = r['timestamp']
    try:
        if r['type']=='trade':
            prior = prior_iso(r)
            if prior == 'in':
                p  = price(r['in_iso'], t)
                assert pd.notna(p) and (p > 0), 'wrong price output: '+r['in_iso']+' '+ str(t)
                v = p * r['in']
            elif prior  == 'out':
                p  = price(r['out_iso'], t)
                assert pd.notna(p) and (p > 0), 'wrong price output: '+r['in_iso']+' '+ str(t)
                v = p * r['out']
            r['value']= v
            r['in_price'] = v / r['in'] if (pd.notna(r['in']) and r['in']!= 0) else 0
            r['out_price'] = v /r['out'] if (pd.notna(r['out']) and r['out']!= 0) else 0
        else:
            if pd.notna(r['in']) and (r['in']>0):
                p = price(r['in_iso'], t)
                if pd.notna(p) and (p > 0):
                    r['in_price'] = p
                    r['value'] = p * r['in']
                else:
                    r['in_price'] = 0
                    r['value'] = 0
            if pd.notna(r['out']) and (r['out']>0):
                p = price(r['out_iso'], t)
                if pd.notna(p) and (p > 0):
                    r['out_price'] = p
                    r['value'] = p * r['out']
                else:
                    r['out_price'] = None
                    r['value'] = None        
        if pd.notna(r['fee']) and (r['fee']>0):
            fp =  price(r['fee_iso'], t)
            if pd.notna(fp) and (fp > 0):
                r['fee_price'] = fp
            else:
                r['fee_price'] = None
        return r
    except Exception as e:
        print(r)
        raise e

Dopo aver avviato la cella seguente, lasciamolo lavorare e andiamo a preparare un caffè.
Può essere conveniente utilizzare il file "fill" nella cartella */src/prices* per riempire il db dei prezzi almeno per le valute più utlizzate, perchè un solo errore (rate limit) attualmente fa crashare tutta la procedura.
Avendo il tempo si dovrebbe rendere più robusto tutto l'algoritmo.


In [ ]:
txs = txs.apply(valorize, axis=1)

In [ ]:
db1.save('valorized', txs)